## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 9/13/2024)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from yfinance import Ticker
from pykalman import KalmanFilter

from src import elt as et

%load_ext autoreload
%autoreload 2

In [140]:
def download(symbol,interval,period):
    stock = Ticker(symbol)
    stock_df = stock.history(interval=interval,
                             period=period,
                             auto_adjust=False,
                             prepost=True, # include aftermarket hours
                            )
    stock_df.columns = stock_df.columns.str.lower().str.replace(' ', '_')
    stock_df.to_pickle(f'./data/{symbol}_{interval}_df.pkl')
    
def load(symbol,interval):
    return pd.read_pickle(f'./data/{symbol}_{interval}_df.pkl')

#########################################
# functions for use to transform tables #
#########################################

# candle parts percentages
def candle_parts_pcts(o, c, h, l):
    full = h - l
    if full == 0:
        # If full is zero, return 0 for all components to avoid division by zero
        return 0, 0, 0
    body = abs(o - c)
    if o > c:
        top_wick = h - o
        bottom_wick = c - l
    else:
        top_wick = h - c
        bottom_wick = o - l
    return top_wick / full, body / full, bottom_wick / full

# previous close and open gap % of pervious candle size
def gap_up_down_pct(o, pc, ph, pl):
    if o == pc:
        return 0
    else:
        return (o - pc) / (ph - pl)
    
    
# z-score calculation
def zscore(x, mu, stdev):
    return (x - mu) / stdev

# direction calculation:
def direction(pctc, mean, stdev):
    
    pct_pos = mean + 0.43073 / 2 * stdev
    pct_neg = mean - 0.43073 / 2 * stdev
    if pctc >= pct_pos:
        return 1
    elif pctc <= pct_neg:
        return 2
    else:
        return 0


def transform(symbol, interval, period):
    
    if load(symbol, interval).shape[0] > 0:
        df = load(symbol, interval)
        
    else:
        download(symbol, interval, period)
        df = load(symbol,interval)
    
    # Kalman filtering (noise reduction algorithm) 
    kf = KalmanFilter(transition_matrices = [1],
                      observation_matrices = [1],
                      initial_state_mean = 0,
                      initial_state_covariance = 1,
                      observation_covariance=1,
                      transition_covariance=0.01
                     )

    state_means, _ = kf.filter(df['adj_close'].values)
    state_means = pd.Series(state_means.flatten(), index=df.index)
    df['kma'] = state_means
    df['sma40'] = df['adj_close'].rolling(window=40).mean().copy()
    df['kma_sma40_diff'] = (df['kma'] - df['sma40']).copy()
    df['kma_sma40_diff_stdev21'] = df['kma_sma40_diff'].rolling(window=21).std().copy()
    df['kma_sma40_diff_mu21'] = df['kma_sma40_diff'].rolling(window=21).mean().copy()

    # Calculate Kalman Filter vs SMA40 difference z-score
    df['kma_sma40_diff_z21'] = df.apply(lambda row: zscore(row['kma_sma40_diff'], row['kma_sma40_diff_mu21'], row['kma_sma40_diff_stdev21']), axis=1, result_type='expand').copy()

    #update 1 day table: candle parts %'s
    df[['pct_top_wick', 'pct_body', 'pct_bottom_wick']] = df.apply(lambda row: candle_parts_pcts(row['open'], row['close'], row['high'],  row['low']), axis=1, result_type='expand').copy()

    #stdev of adjusted close
    df['top_stdev21'] = df['pct_top_wick'].rolling(window=21).std().copy() 
    df['body_stdev21'] = df['pct_body'].rolling(window=21).std().copy() 
    df['bottom_stdev21'] = df['pct_bottom_wick'].rolling(window=21).std().copy()

    #mean of adjusted close
    df['top_mu21'] = df['pct_top_wick'].rolling(window=21).mean().copy() 
    df['body_mu21'] = df['pct_body'].rolling(window=21).mean().copy() 
    df['bottom_mu21'] = df['pct_bottom_wick'].rolling(window=21).mean().copy()

    #z-score of adjusted close
    df['top_z21'] = df.apply(lambda row: zscore(row['pct_top_wick'], row['top_mu21'], row['top_stdev21']), axis=1, result_type='expand').copy()
    df['body_z21'] = df.apply(lambda row: zscore(row['pct_body'], row['body_mu21'], row['body_stdev21']), axis=1, result_type='expand').copy()
    df['bottom_z21'] = df.apply(lambda row: zscore(row['pct_bottom_wick'], row['bottom_mu21'], row['bottom_stdev21']), axis=1, result_type='expand').copy()

    #update 1 day table: % gap btwn current open relative to previous candle size
    df['pc'] = df['close'].shift(1).copy()
    df['ph'] = df['high'].shift(1).copy()
    df['pl'] = df['low'].shift(1).copy()
    df['pct_gap_up_down'] = df.apply(lambda row: gap_up_down_pct(row['open'], row['pc'], row['ph'], row['pl']), axis=1, result_type='expand').copy()

    #stdev of adjusted close
    df['ac_stdev5'] = df['adj_close'].rolling(window=5).std().copy() 
    df['ac_stdev8'] = df['adj_close'].rolling(window=8).std().copy() 
    df['ac_stdev13'] = df['adj_close'].rolling(window=13).std().copy()

    #mean of adjusted close
    df['ac_mu5'] = df['adj_close'].rolling(window=5).mean().copy() 
    df['ac_mu8'] = df['adj_close'].rolling(window=8).mean().copy() 
    df['ac_mu13'] = df['adj_close'].rolling(window=13).mean().copy()

    #z-score of adjusted close
    df['ac_z5'] = df.apply(lambda row: zscore(row['adj_close'], row['ac_mu5'], row['ac_stdev5']), axis=1, result_type='expand').copy()
    df['ac_z8'] = df.apply(lambda row: zscore(row['adj_close'], row['ac_mu8'], row['ac_stdev8']), axis=1, result_type='expand').copy()
    df['ac_z13'] = df.apply(lambda row: zscore(row['adj_close'], row['ac_mu13'], row['ac_stdev13']), axis=1, result_type='expand').copy()

    #target column: direction: -1, 0, 1
    df['adj_close_pctc'] = df['adj_close'].pct_change()
    mean = df['adj_close_pctc'].mean()
    stdev = df['adj_close_pctc'].std()
    df['direction'] = df.apply(lambda row: direction(row['adj_close_pctc'], mean, stdev), axis=1, result_type='expand').copy() 

    #save 1d file for model building
    df[['top_z21', 
        'body_z21', 
        'bottom_z21',
        'top_z21',
        'body_z21',
        'bottom_z21',
        'pct_gap_up_down',
        'ac_z5',
        'ac_z8',
        'ac_z13',
        'kma_sma40_diff_z21',
        'adj_close',
        'direction',
       ]
      ].to_pickle(f'./models/{symbol}_{interval}_model_df.pkl')

In [141]:
download('NVDA', '15m', '1mo')
df = load('NVDA', '15m')
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1281 entries, 2024-08-19 04:00:00-04:00 to 2024-09-16 20:00:00-04:00
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   open          1281 non-null   float64
 1   high          1281 non-null   float64
 2   low           1281 non-null   float64
 3   close         1281 non-null   float64
 4   adj_close     1281 non-null   float64
 5   volume        1281 non-null   int64  
 6   dividends     1281 non-null   float64
 7   stock_splits  1281 non-null   float64
dtypes: float64(7), int64(1)
memory usage: 90.1 KB


In [142]:
df.columns

Index(['open', 'high', 'low', 'close', 'adj_close', 'volume', 'dividends',
       'stock_splits'],
      dtype='object')

In [143]:
transform('NVDA','15m','1mo')

In [144]:
nvda_model_df = pd.read_pickle('./models/NVDA_15m_model_df.pkl')

In [145]:
nvda_model_df.direction.value_counts()

direction
0    441
1    431
2    409
Name: count, dtype: int64

In [146]:
nvda_model_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1281 entries, 2024-08-19 04:00:00-04:00 to 2024-09-16 20:00:00-04:00
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   top_z21             1261 non-null   float64
 1   body_z21            1261 non-null   float64
 2   bottom_z21          1261 non-null   float64
 3   top_z21             1261 non-null   float64
 4   body_z21            1261 non-null   float64
 5   bottom_z21          1261 non-null   float64
 6   pct_gap_up_down     1280 non-null   float64
 7   ac_z5               1277 non-null   float64
 8   ac_z8               1274 non-null   float64
 9   ac_z13              1269 non-null   float64
 10  kma_sma40_diff_z21  1222 non-null   float64
 11  adj_close           1281 non-null   float64
 12  direction           1281 non-null   int64  
dtypes: float64(12), int64(1)
memory usage: 140.1 KB
